In [1]:
# 2019-11-22
# A. Pendleton
# Co-expression network plots -- 3D attempts

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sys
import glob

#Plotting networks
import networkx as nx

In [3]:
# For interactivity
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) #Make it work in notebook mode
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


init_notebook_mode(connected=True)

In [5]:
geneList = ['Sevir.5G058900', 'Sevir.5G254600', 'Sevir.2G052500']

rootDir = '/depot/jwisecav/data/pendlea/coexpression_assessments/development/coexp_development/'


networks = []

#Get a list of the network directories the user has in their Networks path
for directory in glob.glob(rootDir + 'Networks/*/'):
    networkDecay = directory.rsplit('/',2)[1].split('_')[1]
    networks.append(networkDecay)

networksToPlot = []

#Go through each decay value that the user has provided data for (Note. this can be just one decay value/level)
for networkDecay in networks:
    #Find the relevant summary file based on what the decay value is
    networkRootDir =  rootDir + 'Networks/' + 'Network_%s/' % networkDecay
    summaryFile = networkRootDir + 'Module_Summary.txt'
    
    #Parse each line of the summary file
    for line in open(summaryFile, 'r'):
        line=line.rstrip().split('\t') #Strip line and split by tab
        
        #skip headerline
        if '#' in line[0]:
            continue
    
        networkID = line[0] #The network id is column #1
        networkABCFile = networkRootDir + networkID + '.abc' #The edgeweight file is simply networkID.abc in the same directory
        genesInNetwork = line[3].split(' ') #Space-delimited list of genes in the network (so split by space to make a list)
        
        #Go through each gene and see if its in your gene list 
        for gene in genesInNetwork: 
            #If so, then add its relevant network ID to an array that stores the edgeweight files
            #.    for all networks to plot
            if gene in geneList and networkABCFile not in networksToPlot:
                networksToPlot.append(networkABCFile)
                
print('%i networks to plot' % len(networksToPlot))

10 networks to plot


________

# 3D rendering of networks

In [17]:
network = networksToPlot[0]
moduleID = network.rsplit('/',1)[1].replace('.abc','') #extract module ID from the module file

#Define the edgeweight file
edgeWeightFile = network

#Matplotlib defined clearing and size determination of plot
"""fig = plt.figure(figsize=[20,20])
ax = plt.subplot(111)"""

#Create empty graph for networkX
G = nx.Graph()
G = nx.read_weighted_edgelist(edgeWeightFile)
pos=nx.spring_layout(G, dim=3) #This is a randomized layout of the network

labels = pos.keys()

#Edge traces
edge_trace = go.Scatter3d(
    x=[],
    y=[],
    z=[],
    line=dict(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0, z0 = pos[edge[0]]#G.node[edge[0]]['pos']
    x1, y1, z1 = pos[edge[1]] #G.node[edge[1]]['pos']
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])
    edge_trace['z'] += tuple([z0, z1, None])

#NODE TRACES
node_trace = go.Scatter3d(
    x=[],
    y=[],
    z=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        color=[],
        size=20,
        ))

#ADD X,Y POSITIONS OF EACH NODE/GENE
for node in G.nodes():
    x, y, z = pos[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    node_trace['z'] += tuple([z])


#DEFINE COLORS OF NODES
colors = []

for node in G.nodes():
    geneID = node 
    #If gene is in candidate gene list -- then they're blue, else they're gray
    if geneID in geneList:
        colors.append('dodgerblue')
    else:
        colors.append('lightgray')
    #node_info = '# of connections: '+str(len(adjacencies[1]))
    node_trace['text'] +=tuple([geneID])
    
node_trace['marker']['color'] = colors

axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

#DRAW GRAPH
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network %s' % moduleID,
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                plot_bgcolor= 'rgba(0,0,0,0)',
                #xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                #yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))
                scene=dict(
                xaxis=dict(axis),
                yaxis=dict(axis),
                zaxis=dict(axis))))
iplot(fig, filename='networkx')